In [13]:
import requests
import urllib
import datetime
from bs4 import BeautifulSoup
from re import sub
import re
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
from datetime import datetime
import time
from requests.auth import HTTPProxyAuth
import random
import lxml


PREFIX = 'https://www.hltv.org'
proxyDict = None
proxyAuth = None

In [18]:
def get_parsed_page(url, proxy_=None, auth_=None):
    TIMESLEEP = random.uniform(0.25, 0.3)
    time.sleep(TIMESLEEP)
    got_req = requests.get(url, proxies=proxy_, auth=auth_).text
    return BeautifulSoup(got_req)

In [ ]:
# download all urls matches
def get_results_url(filename=None, pages_with_results=[0]):
    all_matches = []
    for i in pages_with_results:
        if i == 0:
            results = get_parsed_page('http://www.hltv.org/results', proxyDict, proxyAuth)
        else:
            results = get_parsed_page(f'http://www.hltv.org/results?offset={i}00', proxyDict, proxyAuth)
        # list of matches
        all_matches += [url_.find('a', {"class": "a-reset"})['href'] for url_ in
                        results.find('div',
                                     {"class": "results-all", 'data-zonedgrouping-group-classes': "results-sublist"}).
                        find_all('div', {"class": "result-con"})]
    all_matches = np.array(all_matches)
    data = pd.DataFrame(all_matches, columns=['match_url'])
    data.index.name = 'id'
    if filename is None:
        return data
    else:
        data.to_csv(f'{filename}.csv')
        return data


get_results_url()

In [ ]:
class MatchPageParams:
    def __init__(self, df, start_index=None, finish_index=None):
        self.df = df
        self.MATCH_PAGE = None
        self.MATCH_ID = None
        self.match = None
        self.start_index = start_index
        self.finish_index = finish_index

    def add_all_params(self):
        time_start = time.time()
        for match in self.df['match_url'][self.start_index:self.finish_index]:
            self.MATCH_ID = list(self.df['match_url']).index(match)
            print(PREFIX + match)
            print('match id -', self.MATCH_ID, '; time spent -', time.time() - time_start)
            self.match = match
            self.add_url_teams()
            self.add_url_event()
            self.add_10_urls_players()
            self.add_total_maps()
            self.add_maps()
            self.add_date()
            self.add_scores()
            self.add_history()
            self.add_ranks()
            self.add_5last_urls_opponents()
            self.add_5last_scores_and_types()
            self.MATCH_PAGE = None
    
    def parse_page(self):
        if self.MATCH_PAGE is None:
            self.MATCH_PAGE = get_parsed_page(PREFIX + self.match, proxyDict, proxyAuth)

    def _get_score_(self):
        self.parse_page()
        score = self.MATCH_PAGE.find('div', {'class': 'match-page'}).\
                find('div', {'class': 'teamsBox'}).\
                find_all('div', {'class': 'team'})
        for i in range(len(score)):
            score[i] = score[i].find('div', {'class': f'team{i + 1}-gradient'}).\
                                find_all('div')[-1].text
            self.df[f'score{i + 1}'][self.MATCH_ID] = score[i]

    def add_scores(self):
        # add columns
        if 'score1' not in self.df.columns:
            self.df['score1'] = ""
        if 'score2' not in self.df.columns:
            self.df['score2'] = ""
        if self.df['score1'][self.MATCH_ID] == "" or self.df['score2'][self.MATCH_ID] == "":
            self._get_score_()

    def _get_total_maps_(self):
        self.parse_page()
        try:
            table = self.MATCH_PAGE.find('div', {'class': 'flexbox-column'}).\
                    find_all('div', {'class': 'mapholder'})
            self.df['total_maps'][self.MATCH_ID] = len(table)
            self.df['maps_played'][self.MATCH_ID] = 0
            for i in table:
                played = i.find('div', {'class': 'results'}).find('div', {'class': 'results-center'})
                if played.text:
                    self.df['maps_played'][self.MATCH_ID] += 1
        except:
            self.df['total_maps'][self.MATCH_ID] = 0
            self.df['maps_played'][self.MATCH_ID] = 0
            

    def add_total_maps(self):
        # add columns
        if 'total_maps' not in self.df:
            self.df['total_maps'] = ""
        if 'maps_played' not in self.df:
            self.df['maps_played'] = ""
        if self.df['total_maps'][self.MATCH_ID] == "" or self.df['maps_played'][self.MATCH_ID] == "":
            self._get_total_maps_()

    def _get_ranks_(self):
        self.parse_page()
        ranks = self.MATCH_PAGE.find('div', {'class': 'lineups'}).\
                find_all('div', {'class': 'box-headline'})
        for i in range(len(ranks)):
            ranks[i] = ranks[i].find('div', {'class': 'teamRanking'}).text
            self.df[f'rank{i + 1}'][self.MATCH_ID] = ranks[i]
            
    def add_ranks(self):
        # add columns
        if 'rank1' not in self.df:
            self.df['rank1'] = ""
        if 'rank2' not in self.df:
            self.df['rank2'] = ""
        if self.df['rank1'][self.MATCH_ID] == "" or self.df['rank2'][self.MATCH_ID] == "":
            self._get_ranks_()
    
    def _get_date_(self):
        self.parse_page()
        self.df['date'][self.MATCH_ID] = datetime.utcfromtimestamp(int(
                self.MATCH_PAGE.find('div', {'class': 'timeAndEvent'}).
                find('div', {'class': 'time'})['data-unix'][:-3])).strftime('%Y-%m-%d')
    
    def add_date(self):
        if 'date' not in self.df:
            self.df['date'] = ""
        if self.df['date'][self.MATCH_ID] == "":
            self._get_date_()

    def _get_history_(self):
        self.parse_page()
        h2h = self.MATCH_PAGE.find('div', {'class': 'head-to-head'}).\
                find_all('div', {'class': 'flexbox-column'})
        self.df['h2h_wins1'][self.MATCH_ID] = h2h[0].find('div', {'class': 'bold'}).text
        self.df['h2h_wins2'][self.MATCH_ID] = h2h[-1].find('div', {'class': 'bold'}).text

    def add_history(self):
        if 'h2h_wins1' not in self.df:
            self.df['h2h_wins1'] = ""
        if 'h2h_wins2' not in self.df:
            self.df['h2h_wins2'] = ""
        if self.df['h2h_wins1'][self.MATCH_ID] == "" or self.df['h2h_wins2'][self.MATCH_ID] == "":
            self._get_history_()

    def _get_5last_score_type_(self, team_id, match_id):
        self.parse_page()
        try:
            tmp = self.MATCH_PAGE.find('div', {'class': 'past-matches'}).\
                    find_all('div', {'class': 'half-width'})[team_id].find('table', {'class': 'matches'}).\
                    find_all('tr', {'class': 'table'})[match_id]
            total = tmp.find('a', {'data-link-tracking-page': 'Matchpage'}).text
            self.df[f'5last_match{match_id + 1}_total_maps{team_id + 1}'][self.MATCH_ID] = total
            score = tmp.find('td', {'class': 'result'}).text.split(' - ')
            if int(score[0]) > int(score[1]):
                score = [1, 0]
            else:
                score = [0, 1]
            self.df[f'5last_match{match_id + 1}_score{team_id + 1}'][self.MATCH_ID] = score[0]
            self.df[f'5last_match{match_id + 1}_opponent_score{team_id + 1}'][self.MATCH_ID] = score[1]
        except:
            self.df[f'5last_match{match_id + 1}_total_maps{team_id + 1}'][self.MATCH_ID] = 0
            self.df[f'5last_match{match_id + 1}_score{team_id + 1}'][self.MATCH_ID] = 0
            self.df[f'5last_match{match_id + 1}_opponent_score{team_id + 1}'][self.MATCH_ID] = 0
        
    
    def add_5last_scores_and_types(self):
        for team_id in range(2):
            for match_id in range(5):
                if f'5last_match{match_id + 1}_total_maps{team_id + 1}' not in self.df:
                    self.df[f'5last_match{match_id + 1}_total_maps{team_id + 1}'] = ""
                if f'5last_match{match_id + 1}_score{team_id + 1}' not in self.df:
                    self.df[f'5last_match{match_id + 1}_score{team_id + 1}'] = ""
                if f'5last_match{match_id + 1}_opponent_score{team_id + 1}' not in self.df:
                    self.df[f'5last_match{match_id + 1}_opponent_score{team_id + 1}'] = ""
                if self.df[f'5last_match{match_id + 1}_total_maps{team_id + 1}'][self.MATCH_ID] == "" or\
                    self.df[f'5last_match{match_id + 1}_score{team_id + 1}'][self.MATCH_ID] == "" or\
                    self.df[f'5last_match{match_id + 1}_opponent_score{team_id + 1}'][self.MATCH_ID] == "":
                    self._get_5last_score_type_(team_id, match_id)

    def _get_5last_opponents_(self, team_id):
        self.parse_page()
        arr = []
        tmp = self.MATCH_PAGE.find('div', {'class': 'past-matches'}).\
                find_all('div', {'class': 'half-width'})[team_id].find('table', {'class': 'matches'}).\
                find_all('tr', {'class': 'table'})
        for i in tmp:
            arr += [i.find('td', {'class': 'opponent'}).find('a')['href']]
        self.df[f'opponents_url_history_team{team_id + 1}'][self.MATCH_ID] = arr
        
    
    def add_5last_urls_opponents(self):
        for team_id in range(2):
            if f'opponents_url_history_team{team_id + 1}' not in self.df:
                self.df[f'opponents_url_history_team{team_id + 1}'] = ""
            if self.df[f'opponents_url_history_team{team_id + 1}'][self.MATCH_ID] == "":
                self._get_5last_opponents_(team_id)

    def _get_players_url__(self, team_id):
        self.parse_page()
        arr = []
        tmp = self.MATCH_PAGE.find('div', {'class': 'lineups'}).\
                find_all('div', {'class': 'lineup'})[team_id].find('div', {'class': 'players'}).\
                find_all('td', {'class': 'player'})
        for i in tmp:
            try:
                arr += [i.find('a')['href']]
            except:
                arr += [""]
        self.df[f'players_url_{team_id + 1}'][self.MATCH_ID] = arr[:5]
        #print(arr)

    def add_10_urls_players(self):
        for team_id in range(2):
            if f'players_url_{team_id + 1}' not in self.df:
                self.df[f'players_url_{team_id + 1}'] = ""
            if self.df[f'players_url_{team_id + 1}'][self.MATCH_ID] == "":
                self._get_players_url__(team_id)

    def _get_event_url_(self):
        self.parse_page()
        self.df['event_url'][self.MATCH_ID] = self.MATCH_PAGE.find('div', {'class': 'timeAndEvent'}).\
                find('div', {'class': 'event'}).find('a')['href']
    
    def add_url_event(self):
        if 'event_url' not in self.df:
            self.df['event_url'] = ""
        if self.df['event_url'][self.MATCH_ID] == "":
            self._get_event_url_()
    
    def _get_maps_(self):
        self.parse_page()
        arr = []
        map_name = []
        score_maps1 = []
        score_maps2 = []
        picks = []
        try:
            tmp = self.MATCH_PAGE.find('div', {'class': 'flexbox-column'}).\
                    find_all('div', {'class': 'mapholder'})
            for i in tmp:
                played = i.find('div', {'class': 'results'}).find('div', {'class': 'results-center'})
                if played.text:
                    arr += [played.find('a')['href']]
                    map_name += [i.find('div', {'class': 'map-name-holder'}).find('div', {'class': 'mapname'}).text]
                    table = i.find('div', {'class': 'results-left'})
                    if self.df['total_maps'][self.MATCH_ID] > 1:
                        if table['class'][-1] == 'pick':
                            picks += [1]
                    score_maps1 += [table.find('div', {'class': 'results-team-score'}).text]
                    table = i.find('span', {'class': 'results-right'})
                    if self.df['total_maps'][self.MATCH_ID] > 1:
                        if table['class'][-1] == 'pick':
                            picks += [-1]
                    score_maps2 += [table.find('div', {'class': 'results-team-score'}).text]
            self.df['maps_url'][self.MATCH_ID] = arr
            self.df['maps_name'][self.MATCH_ID] = map_name
            self.df['score1_maps'][self.MATCH_ID] = score_maps1
            self.df['score2_maps'][self.MATCH_ID] = score_maps2
            if not picks:
                self.df['picks'][self.MATCH_ID] = "None"
            else:
                self.df['picks'][self.MATCH_ID] = picks
        except Exception as e:
            print(e)
            self.df['maps_url'][self.MATCH_ID] = "None"
            self.df['maps_name'][self.MATCH_ID] = "None"
            self.df['score1_maps'][self.MATCH_ID] = "None"
            self.df['score2_maps'][self.MATCH_ID] = "None"
            self.df['picks'][self.MATCH_ID] = "None"
    
    def add_maps(self):
        if 'maps_url' not in self.df:
            self.df['maps_url'] = ""
        if 'maps_name' not in self.df:
            self.df['maps_name'] = ""
        if 'score1_maps' not in self.df:
            self.df['score1_maps'] = ""
        if 'score2_maps' not in self.df:
            self.df['score2_maps'] = ""
        if 'picks' not in self.df:
            self.df['picks'] = ""
        if self.df['maps_url'][self.MATCH_ID] == "" or self.df['maps_name'][self.MATCH_ID] == "" or\
                self.df['score1_maps'][self.MATCH_ID] == "" or self.df['score2_maps'][self.MATCH_ID] == "" or\
                self.df['picks'][self.MATCH_ID] == "":
            self._get_maps_()
    
    def _get_url_teams_(self):
        self.parse_page()
        tmp = self.MATCH_PAGE.find('div', {'class': 'teamsBox'}).\
                find_all('div', {'class': 'team'})
        self.df['url_team1'][self.MATCH_ID] = tmp[0].find('a')['href']
        self.df['url_team2'][self.MATCH_ID] = tmp[1].find('a')['href']
    
    def add_url_teams(self):
        if 'url_team1' not in self.df:
            self.df['url_team1'] = ""
        if 'url_team2' not in self.df:
            self.df['url_team2'] = ""
        if self.df['url_team1'][self.MATCH_ID] == "" or self.df['url_team2'][self.MATCH_ID] == "":
            self._get_url_teams_()


MatchPageParams(df).add_all_params()